In [1]:
import pandas as pd

#财务收款数据和财务退款数据 的清洗
def clean_data(path):
    data = pd.read_csv(path,engine='python')
    for column in data.columns:
        try:
            data[column]=data[column].str.replace('\t','')
        except:
            pass
    data['设备名称'] = data['设备名称'].str.replace('（','(')
    data['设备名称'] = data['设备名称'].str.replace('）',')')
    return data

In [2]:
#读取人员名单
employee = pd.read_excel(r'D:\周报制作\人员名单.xlsx')[['city_employee','城市','区域','部门']]
#读取财务收款数据
data_in = clean_data(r'D:\周报制作\报表-收款数据20180810_12-55-39.csv')

In [3]:
data_in['订单号'] = data_in['订单号'].astype(str)
data_in['city_employee'] = data_in['销售人城市'] + '/' + data_in['销售人']
#匹配销售人城市、区域、部门
data_in_add_department = pd.merge(data_in,employee,left_on='city_employee',right_on='city_employee',how='left')
#未匹配到的人员名单
employee_wait_add = data_in_add_department[data_in_add_department['部门'].isnull()==True][['销售人','销售人城市','city_employee','部门']].drop_duplicates()
print(employee_wait_add.shape)
print(employee_wait_add)

(0, 4)
Empty DataFrame
Columns: [销售人, 销售人城市, city_employee, 部门]
Index: []


In [4]:
#data_in_merge 订单折叠
data_in_merge = data_in_add_department.drop_duplicates(subset='订单号',keep='first',inplace=False)
three_departments = ['直销部','口碑项目部','信息流程优化部']
data_in_merge_need = data_in_merge[data_in_merge['部门'].isin(three_departments)==True]
#城市、门店 去重
city_door_list = data_in_merge_need.drop_duplicates(subset=['销售人城市','商家ID'],keep='first',inplace=False)
city_door_count = city_door_list['销售人城市'].value_counts()
#城市、门店数统计
city_door_count

上海     93
广州     79
成都     57
北京     53
深圳     45
杭州     44
南京     37
武汉     36
厦门     27
西安     26
沈阳     25
苏州     24
重庆     24
天津     22
青岛     21
郑州     16
宁波     16
长沙     15
无锡     14
南宁     14
济南     14
福州     13
合肥     11
湖州     11
哈尔滨    11
常州     10
大连      9
佛山      5
温州      4
南昌      2
Name: 销售人城市, dtype: int64

In [5]:
city_brand_list = city_door_list.drop_duplicates(subset=['品牌名称','销售人城市'],keep='first',inplace=False)
city_brand_count = city_brand_list['销售人城市'].value_counts()
city_brand_count

上海     79
广州     53
成都     49
深圳     44
北京     43
杭州     42
南京     33
武汉     27
厦门     23
苏州     22
西安     22
重庆     22
青岛     20
天津     20
沈阳     19
宁波     16
郑州     16
无锡     14
南宁     14
长沙     14
福州     13
济南     13
合肥     11
湖州     11
常州     10
大连      9
哈尔滨     8
佛山      5
温州      4
南昌      2
Name: 销售人城市, dtype: int64

In [6]:
writer = pd.ExcelWriter(r'D:\周报制作\财务收款门店数与品牌数\20180810_财务收款门店数与品牌数.xlsx')
data_in_add_department.to_excel(writer,'财务收款订单',index=False)
data_in_merge_need.to_excel(writer,'财务收款订单折叠',index=False)
city_door_list.to_excel(writer,'去重后的门店',index=False)
city_brand_list.to_excel(writer,'去重后的品牌',index=False)
city_door_count.to_excel(writer,'门店数统计',index_label='城市')
city_brand_count.to_excel(writer,'品牌数统计',index_label='城市')
writer.save()